In [1]:
import numpy as np
import random

In [9]:
def make_key_pub(G,F,n,k,l):   
    A = random_matrix(F, l, k-l)
    I_l = identity_matrix(F,l)
    E = zero_matrix(F, l, n)
    d = vector(F, n) 
    for i in range(W):
        ind = random.randint(0, n - 1)
        d[ind] = 1
    for i in range(l):
        for j in range(n):
            if d[j] == 1:
                E[i, j] = F.random_element()
    
    P = A.augment(I_l)*G
    E_pub = P + E
    return E_pub, A, E

In [10]:
def encryption(m,G,E_pub,n,l):
    e = zero_matrix(F,1,n)
    for i in range(w):
        ind = random.randint(0,n-1)
        e[0,ind] = 1
    alpha = random_matrix(F,1,l)
    y = m*G + alpha*E_pub + e
    return y, alpha

In [11]:
def welsh_berlekamp(y_new, G_new, n, k, l):
    n1 = G_new.ncols()
    k1 = G_new.nrows()
    d = n1 - k1 + 1
    deg_N = k1 - 1 + (d-1) // 2
    deg_L = (d-1) // 2
    a_ = matrix(F, 1, n1)    # новый вектор a', в котором элементы совпадают со второй строчкой матрицы G'
    for i in range(n1):
        a_[0,i] = G_new[1,i]
    N = zero_matrix(F, n1, deg_N+1)                
    for i in range(n1):
        for j in range(deg_N+1):
            N[i,j]= a_[0,i]^j
    L=zero_matrix(F,n1,deg_L+1)
    for i in range(n1):
        for j in range(deg_L+1):
            L[i,j] = y_new[0,i]*a_[0,i]^j
    M = N.augment(-1*L)
    sol = M.right_kernel().random_element()
    N_coefs = sol[0:deg_N+1]
    L_coefs = sol[deg_N+1:]
    R.<x> = F[]
    N = sum([N_coefs[i] * x^i for i in range(deg_N+1)])
    L = sum([L_coefs[i] * x^i for i in range(deg_L+1)])
    if(L.is_zero()):
        return zero_matrix(F,1,k)
    U = N//L
    m_ = vector(F,U.list())
    return m_

In [12]:
def decryption(y, G, alpha, A, E, n, k ,l):
    cols_ind = []
    zero_col = zero_matrix(F,l,1)
    for i in range(n):
        if E[:,i]!=zero_col:
            cols_ind.append(i)
    G_new = G[:,[i for i in range(n) if i not in cols_ind]]
    y_new = y[:,[i for i in range(n) if i not in cols_ind]]
    m_ = welsh_berlekamp(y_new, G_new, n, k, l)
    message = zero_matrix(F, 1, k-l)
    for i in range(k-l):
        message[0,i] = m_[i]
    origin_message = message-alpha*A
    return origin_message

In [13]:
def square(G):
    N = G.nrows()
    res = zero_matrix(F, N*(N-1)//2 + N, G.ncols())
    r = 0
    for i in range(N):
        for j in range(i, N):
            res[r] = G[i].pairwise_product(G[j])
            r += 1
    return res

In [14]:
def attack(y, G, alpha, E_pub, A, n, k, l):
    cols_ind = []
    G_ = block_matrix([[G],[E_pub]])
    for j in range (n):
        if(square(G_[:,  [i for i in range(n) if i != j]]).rank() != square(G_).rank()):
            cols_ind.append(j)
    G_new = G[:,[i for i in range(n) if i not in cols_ind]]
    y_new = y[:,[i for i in range(n) if i not in cols_ind]]
    m_ = decode(y_new, G_new, n, k, l)
    message = zero_matrix(F, 1, k-l)
    for i in range(k-l):
        message[0,i] = m_[i]
    origin_message = message-alpha*A
    return origin_message

In [15]:
F = GF(17)
n = 17
k = 6
l = 2
W = 4
w = 3
elements = list(F)
ksi = vector(F, random.sample(elements, n)) # Носитель RS-кода
G = zero_matrix(F,k,n)
for i in range(k):
    for j in range(n):
        G[i,j] = ksi[j]^i
G # Порождающая матрица

[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1]
[15  5 16  0 13  4  3 10  7  2  6 14  1 12 11  8  9]
[ 4  8  1  0 16 16  9 15 15  4  2  9  1  8  2 13 13]
[ 9  6 16  0  4 13 10 14  3  8 12  7  1 11  5  2 15]
[16 13  1  0  1  1 13  4  4 16  4 13  1 13  4 16 16]
[ 2 14 16  0 13  4  5  6 11 15  7 12  1  3 10  9  8]

In [16]:
E_pub, A, E = make_key_pub(G,F,n,k,l)
E_pub # Публичный ключ

[ 5  2 15  9  3  6  8  0  9 16 13  2  5 11 13  6 10]
[16  1 16  5  3 12  4  6 10  1  8 13  8  3 13  2  3]

In [17]:
E # Матрица большой ошибки

[ 0  0  0  0  1  5  0  0  0  5  0  0  0  3  0  0  0]
[ 0  0  0  0  7 12  0  0  0  2  0  0  0  1  0  0  0]

In [19]:
m = zero_matrix(F,1,k)
for i in range(k-l):
    m[0,i] = F.random_element()
m # исходное сообщение длины k + l нулей

[12 11  1  5  0  0]

In [20]:
y, alpha = encryption(m,G,E_pub,n,l)
y # шифртекст и значения l элементов, которые были подставлены в конец сообщения m

[ 3  9  9  5 13  3  9  3 10  7 10 12 10  2 12  5  8]

In [185]:
message = decryption(y, G, alpha, A, E, n, k ,l)
message, m[:, :k-l] # успешное декодирование легального пользователя

([15 16  5  7], [15 16  5  7])

In [218]:
message = attack(y,G,alpha,E_pub,A,n,k,l)
message, m[0, :k-l] # Успешно восстановленное сообщение после атаки

([15 16  5  7], [15 16  5  7])